In [1]:
cd ../src

/home/user/Desktop/Kaggle/Script/NLP/FPrize Effective Arguments/src


In [3]:
import torch
import random
import numpy as np
import pandas as pd
from pathlib import Path
# from utils.viz import visualize

from transformers import DataCollatorWithPadding
from transformers import AutoTokenizer, AutoModel, AutoConfig

%env TOKENIZERS_PARALLELISM = true


import warnings
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 500)
warnings.simplefilter("ignore")

env: TOKENIZERS_PARALLELISM=true


test/ - A folder containing an example essay from the test set. The actual test set comprises about 3,000 essays in a format similar to the training set essays.
The test set essays are distinct from the training set essays.

# Params

In [4]:
ROOT_DATA = Path('../data/')
data_folder = ROOT_DATA/"feedback-prize-effectiveness"

# Data

In [5]:
train_df = pd.read_csv(ROOT_DATA/'feedback-prize-2021'/"train.csv")
train_df.shape

(144293, 8)

In [6]:
train_df.head(2)

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59


In [7]:
train_df['essay_id'] = train_df['id'] if "essay_id" not in train_df.columns else train_df['essay_id']

In [8]:
fold_df = pd.read_csv(data_folder/"df_folds.csv")
fold_df.shape

(4191, 7)

In [9]:
fold_df.head(2)

,essay_id,fold_k_5_seed_42,fold_k_5_seed_2020,fold_k_8_seed_42,fold_k_8_seed_2020,fold_k_10_seed_42,fold_k_10_seed_2020
0,00066EA9880D,2,3,0,6,4,8
1,000E6DE9E817,2,1,5,3,4,9


In [10]:
train_df = train_df.merge(fold_df,how='left',on="essay_id")
train_df.shape

(144293, 15)

In [11]:
train_df.head(2)

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring,essay_id,fold_k_5_seed_42,fold_k_5_seed_2020,fold_k_8_seed_42,fold_k_8_seed_2020,fold_k_10_seed_42,fold_k_10_seed_2020
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...,423A1CA112E2,4.0,2.0,5.0,4.0,8.0,8.0
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59,423A1CA112E2,4.0,2.0,5.0,4.0,8.0,8.0


In [12]:
for c in ['fold_k_5_seed_42']:
    train_df[c] = train_df[c].fillna(20).astype(int)

In [13]:
train_df[c].value_counts()

20    107528
4       7441
0       7381
1       7343
2       7324
3       7276
Name: fold_k_5_seed_42, dtype: int64

In [14]:
train_df.loc[train_df[c]!=20,c] = 30

In [14]:
# train_df.discourse_effectiveness.value_counts()

In [13]:
classe_mapper = {'Ineffective':0,"Adequate":1,"Effective":2}
classe_mapper

{'Ineffective': 0, 'Adequate': 1, 'Effective': 2}

In [14]:
# train_df['target'] = train_df['discourse_effectiveness'].map(classe_mapper)

In [15]:
# train_df['target'] = 1 if "discourse_effectiveness" not in train_df.columns else train_df['discourse_effectiveness'].map(classe_mapper)

In [16]:
# train_df.target.value_counts()

In [17]:
class args:
    seed = 2022
    
    # Model
    model_name = "microsoft/deberta-v3-large" #"funnel-transformer/large" #"allenai/longformer-large-4096"
    
    # CV
    kfold_name = "fold_k_5_seed_42"
    selected_folds = [0,1,2,3,4]
    
    # Paths
    data_folder = ROOT_DATA/'feedback-prize-2021'
    name = "deberta_v3_large"
    checkpoints_path = Path(fr'../checkpoint/MLM/{name}/')
    pretrained_path = Path(fr'../checkpoint/MLM/{name}/glob/')
    add_special_tokens = True
    input_type = "cls_end"
    sample = False
    mask_pct = 0.15
    use_dropout = True
    use_gradient_checkpointing = True
    # Names
    checkpoints_name = 'log'   
    
    model = {
            "max_len":1024,
            'loss':"nn.CrossEntropyLoss",
            'pretrained_config':Path(fr'../checkpoint/MLM/{name}/config.pth'),
            "pretrained_weights":None,
            "pretrained_tokenizer":Path(fr'../checkpoint/MLM/{name}/tokenizer'),
            "num_labels":3,
            "model_name":model_name
            }
    
    optimizer = {
            "name":"optim.AdamW",
            'params':{"lr":4e-6,'eps':1e-6,"betas":[0.9, 0.999],
                     "weight_decay": 0.01
                     },            
            }

    scheduler = {
            "name":"poly",
            'params':{
                      "lr_end":1e-7,"power":3,"epochs":10
                     },
            "warmup":0.1,            
            }
    
    train_loader = {
            "batch_size":1,
            'drop_last':True,
            "num_workers":15,
            "pin_memory":False,
            "shuffle":True,
            }
    
    val_loader = {
            "batch_size":1,
            'drop_last':False,
            "num_workers":15,
            "pin_memory":False,
            "shuffle":False
            }
    trainer = {"use_amp":False,'epochs':10}
    callbacks = {'save':True,"es":False,"patience":0,
                 'verbose_eval':1,"epoch_pct_eval":5/5,"epoch_eval_dist":"uniforme",
                 "metric_track":"val_loss","mode":"min",'top_k':1,"softmax_before":0
                }
    
    
    device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
#     device = torch.device("cpu")
    
args.checkpoints_path.mkdir(parents=True,exist_ok=True)

In [18]:
from train_utils_mlm import kfold,FeedbackModel

In [19]:
kfold(args,train_df)

----------- fold_k_5_seed_42 ---------

-------------   Fold 1 / 6  -------------



  0%|          | 0/14756 [00:00<?, ?it/s]

  0%|          | 0/14756 [00:00<?, ?it/s]

  0%|          | 0/838 [00:00<?, ?it/s]

  0%|          | 0/838 [00:00<?, ?it/s]

...

Using Pretrained Weights
../checkpoint/MLM/deberta_v3_large/glob/global_val_log_loss=1.8484 .pth
reseizing in nn
Using Pretrained Weights
../checkpoint/MLM/deberta_v3_large/glob/global_val_log_loss=1.8484 .pth
    -> 565142544 trainable parameters



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/14760 [00:00<?, ?it/s]

[MASK] [MASK] ▁Under ▁the ▁elect [MASK] ▁college [MASK] , ▁voters ▁vote ▁not ▁for ▁the [MASK] , [MASK] ▁for ▁a ▁slate ▁of ▁electors , ▁who ▁in [MASK] ▁elect ▁the ▁President . [MASK] [MASK] ▁If ▁the [MASK] ▁are ▁indeed ▁in titled [MASK] ▁their ▁opinion ▁and ▁they ▁a [MASK] ve ▁the ▁power [MASK] ▁come ▁the [MASK] ▁has ▁yet ▁to ▁even [MASK] ▁about ▁changing ▁the ▁elect orial [MASK] ▁process . ▁If [MASK] ▁people ▁do ▁in fact ▁get ▁to [MASK] ▁for ▁which ▁ever ▁President ▁they ▁want , [MASK] [MASK] ▁come [MASK] ksha ▁vote ▁for ▁someone ▁to ▁vote ▁for ▁them , ▁not ▁even ▁considering ▁that ▁that ▁person ▁that ▁they ▁voted ▁for ▁just ▁might ▁defy ▁dork ▁will ▁of ▁the ▁people ▁and ▁choose ▁the ▁other ▁candidate . [MASK] 
 
 [MASK] ▁The ▁single ▁best ▁argument [MASK] ▁the ▁el ce torial ▁college ▁is ▁what [MASK] ▁might ▁call ▁the ▁disaster ▁factor . [MASK] ▁' ▁vegetables [MASK] [MASK] ▁in ▁1960 ▁segregation [MASK] ▁in ▁the [MASK] ian [MASK] ▁legislature [MASK] ▁succeeded ▁in ▁replacing [MASK] ▁new

KeyboardInterrupt: 